In [12]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [13]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [14]:
logger.remove()
logger.add(sys.stderr, level="INFO")

2

In [15]:
#! Set these variables
work_dir_name = "pituitary1"
label_prefix = "pituitary"

I think I have function for part of the below

In [18]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
drive_root = Path("/media/smbshare")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"

winroot = Path(r"H:\3Tpioneer_bids")
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output" / "3Tpioneer_bids"


Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [19]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("pituitary")
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-10-24 18:10:39.537 | INFO     | monai_training.preprocess:prepare_labels:164 - Prepare Labels
100%|██████████| 13/13 [00:00<00:00, 385.59it/s]
2024-10-24 18:10:39.572 | INFO     | monai_training.preprocess:prepare_labels:226 - Dataset size: 13


In [20]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    print("Scan: ", scan.subid, scan.label_path)
    print("Prediction: ", inference.subid, inference.label_path)

Scan:  1029 /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pituitary.nii.gz
Prediction:  1029 /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1029/ses-20170816/t1_ensemble.nii.gz
Scan:  1037 /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/pituitary.nii.gz
Prediction:  1037 /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1037/ses-20210508/t1_ensemble.nii.gz
Scan:  1052 /media/smbshare/3Tpioneer_bids/sub-ms1052/ses-20180803/pituitary.nii.gz
Prediction:  1052 /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1052/ses-20180803/t1_ensemble.nii.gz
Scan:  1057 /media/smbshare/3Tpioneer_bids/sub-ms1057/ses-20181212/pituitary.nii.gz
Prediction:  1057 /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1057/ses-20181212/t1_ensemble.nii.gz
Scan:  1103 /media/smbshare/3Tpioneer_bids/sub-ms1103/ses-201912

Calculate Dice Scores

In [25]:
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.label_path).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    print("{}: {:0.2f}".format(scan.subid, score))

Dice Scores
1029: 0.83
1037: 0.85
1052: 0.78
1057: 0.79
1103: 0.77
1111: 0.90
1207: 0.82
1224: 0.80
1241: 0.77
1248: 0.80
1323: 0.89
1401: 0.78
1492: 0.85


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [26]:
for inference in ensemble_out_dataset:
    label_path = inference.label_path
    new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
    utils.set_label_value(label_path, new_label_path, 2)

Produce commands to open images and labels in itksnap

In [27]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    flair = scan.with_root()
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.root / new_label_path.name
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1029/ses-20170816/t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pituitary.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1037/ses-20210508/t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/pituitary.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1052/ses-20180803/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1052/ses-20180803/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1052/ses-20180803/t1_ensemble_val2.ni

Produce commands to open images and labels in freeview

In [23]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat